In [1]:
import numpy as np
import tifffile as tiff

def linear_interpolate(volume, mask):
    """
    Linearly interpolate missing B-scans along axis 0.
    
    Args:
        volume (np.ndarray): Corrupted OCTA volume (D, H, W)
        mask (np.ndarray): Binary mask (D,) where 1 = missing, 0 = present
    
    Returns:
        np.ndarray: Inpainted volume
    """
    filled = volume.copy()
    D = volume.shape[0]
    known_indices = np.where(mask == 0)[0]
    missing_indices = np.where(mask == 1)[0]

    for idx in missing_indices:
        lower_known = known_indices[known_indices < idx]
        upper_known = known_indices[known_indices > idx]

        if len(lower_known) == 0 or len(upper_known) == 0:
            # Use nearest known slice if interpolation is not possible
            nearest = lower_known[-1] if len(lower_known) else upper_known[0]
            filled[idx] = volume[nearest]
        else:
            lower = lower_known[-1]
            upper = upper_known[0]
            alpha = (idx - lower) / (upper - lower + 1e-5)
            filled[idx] = (1 - alpha) * volume[lower] + alpha * volume[upper]

    return filled

def load_volume_and_mask(volume_path, mask_path):
    volume = tiff.imread(volume_path)
    mask_volume = tiff.imread(mask_path)
    if mask_volume.ndim == 3:
        mask = (mask_volume[:, 0, 0] > 0).astype(np.uint8)
    else:
        mask = mask_volume.astype(np.uint8)
    return volume, mask

def save_inpainted_volume(volume, out_path):
    tiff.imwrite(out_path, volume.astype(np.uint16))

# === Pipeline Execution ===

# File paths
corrupted_path = '/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_corrupted.tif'
mask_path = '/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_mask.tif'
output_path = '/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_corrupted_LinearInterp.tif'

# Step 1: Load corrupted volume and mask
volume, mask = load_volume_and_mask(corrupted_path, mask_path)

# Step 2: Inpaint with linear interpolation
inpainted_volume = linear_interpolate(volume, mask)

# Step 3: Save as TIF (no metadata)
save_inpainted_volume(inpainted_volume, output_path)

print(f"Inpainted volume saved to: {output_path}")


Inpainted volume saved to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_corrupted_LinearInterp.tif
